# Import Packages

In [1]:
import pandas as pd #<- package used to import and organize data
import numpy as np #<- package used to import and organize data
import math
import os #<- package used to work with system file paths
import seaborn as sns #<- package used to plot graphs
from matplotlib import pyplot as plt #<- another package used to plot graphs
from itertools import cycle #<- package used to iterate down rows (used in step 5 to add tap column)
import ipywidgets as widgets #<- widget tool to generate button and tab for graphs
from IPython.display import display #<- displays widgets
from ipyfilechooser import FileChooser
import pingouin as pg
from pingouin import ttest
from scipy.stats import ttest_ind
print("done step 1")

done step 1


# Designate Directory

In [ ]:
starting_directory = '/Users/Joseph'
chooser = FileChooser(starting_directory)
display(chooser)

In [ ]:
print(chooser.selected_path)
folder_path=chooser.selected_path

# To Read In Tap Response Data:

### Setting Tap Paramaters Here...

In [3]:
number_of_taps = 30 ###<------ Taps in your experiment.

number_taps = range(1, number_of_taps + 1)  #<- do not change this


# if you have different ISIs in the same folder, then come back and change this 
# when you are graphing for the second set of data with the other ISI 
# (Generally data from same ISIs are graphed together)
# If changing ISI mid-analysis, you can just skip straight to step 8 after running this cell again

ISI = 10  ### <- What is your ISI? change accordingly
first_tap = 600 ### <- when is your first tap? check your TRV files

#Here, open up one of the trv files to determine the times for each of these taps. 
lower = np.arange(first_tap-1, first_tap-1+(number_of_taps*ISI), ISI) #(first tap, last tap+10s, ISI)
upper = np.arange(first_tap+1, first_tap+1+(number_of_taps*ISI), ISI) #(first tap, last tap+10s, ISI)
tolerances = list(zip(lower, upper)) 
taps = [i for i in range(1,number_of_taps+1)]

#### Add 31st Tap Here --------------------------------------------------<------------
tolerances.append((1188,1191))
taps.append(31)


#assign each tolerance to a tap number
accurate_taps = list((zip(taps, tolerances)))
print(accurate_taps)

for pair in accurate_taps:
   tap = pair[0]
   tolerance = pair[1]
   print("tap "+str(tap), "tolerance="+str(tolerance))

print("done step 3")

[(1, (599, 601)), (2, (609, 611)), (3, (619, 621)), (4, (629, 631)), (5, (639, 641)), (6, (649, 651)), (7, (659, 661)), (8, (669, 671)), (9, (679, 681)), (10, (689, 691)), (11, (699, 701)), (12, (709, 711)), (13, (719, 721)), (14, (729, 731)), (15, (739, 741)), (16, (749, 751)), (17, (759, 761)), (18, (769, 771)), (19, (779, 781)), (20, (789, 791)), (21, (799, 801)), (22, (809, 811)), (23, (819, 821)), (24, (829, 831)), (25, (839, 841)), (26, (849, 851)), (27, (859, 861)), (28, (869, 871)), (29, (879, 881)), (30, (889, 891)), (31, (1188, 1191))]
tap 1 tolerance=(599, 601)
tap 2 tolerance=(609, 611)
tap 3 tolerance=(619, 621)
tap 4 tolerance=(629, 631)
tap 5 tolerance=(639, 641)
tap 6 tolerance=(649, 651)
tap 7 tolerance=(659, 661)
tap 8 tolerance=(669, 671)
tap 9 tolerance=(679, 681)
tap 10 tolerance=(689, 691)
tap 11 tolerance=(699, 701)
tap 12 tolerance=(709, 711)
tap 13 tolerance=(719, 721)
tap 14 tolerance=(729, 731)
tap 15 tolerance=(739, 741)
tap 16 tolerance=(749, 751)
tap 17 to

### Constructing .trv filelist

In [ ]:
os.chdir(folder_path) #<- setting your working directory so that your images will be saved here

filelist = list() #<- empty list
for root, dirs, files in os.walk(folder_path): #<- this for loop goes through your folder 
    for name in files:
        if name.endswith('.trv'): #<- and takes out all files with a .trv (file that contains your data)
            filepath = os.path.join(root, name) #<- Notes down the file path of each data file
            filelist.append(filepath) #<- saves it into the list
    
print(filelist)
print('done step 4')

### generating list of strains (groups)

In [ ]:
genotype=[]
for f in filelist:
    genotype.append(f.split('/')[6])
# print(np.unique(genotype))
genotypes=np.unique(genotype)
# print(genotypes)
# print(len(np.unique(genotypes)))
strainNs = enumerate(genotypes,1)
strain_enum=list(strainNs)
print(strain_enum)
StrainNames=dict(strain_enum)
print(StrainNames)

### Defining functions to reading in data

In [ ]:
def Tap_ProcessData(strain): #<- an example of a user-defined function
    strain_filelist = [x for x in filelist if strain in x] #<- goes through the list and filters for keyword
    Strain_N = len(strain_filelist) #<- N per strain, or number of plates
    Plate_N = 1
    print(f'this strain/treatment has {Strain_N} plates') #<- will output as the first number
    if Strain_N == 0:
        raise AssertionError ('{} is not a good identifier'.format(strain))
    else:
        pass
#     for f in strain_filelist:
#         DF_Total = pd.concat(pd.read_csv(f, sep=' ', skiprows = 4, header = None))
        DF_Total = pd.concat([pd.read_csv(f, sep=' ', header = None, encoding_errors='ignore') for f in strain_filelist],
                      ignore_index=True) #<- imports your data files
#         DF_Total = DF_Total.dropna(axis = 1) #<- cleans your data
        DF_Total = DF_Total.rename( #<- more cleaning
                    {0:'time',
                    2:'rev_before',
                    3:'no_rev',
                    4:'stim_rev',
                    7:'dist',
                    8:'dist_std',
                    9:'dist_stderr',
                    11:'dist_0th',
                    12:'dist_1st',
                    13:'dist_2nd',
                    14:'dist_3rd',
                    15:'dist_100th',
                    18:'dura',
                    19:'dura_std',
                    20:'dura_stderr',
                    22:'dura_0th',
                    23:'dura_1st',
                    24:'dura_2nd',
                    25:'dura_3rd',
                    26:'dura_100th'}, axis=1)
        #check function here for NaN Columns
        DF_Total['plate'] = 0
        DF_Total['prob'] = DF_Total['stim_rev']/ (DF_Total['no_rev'] + DF_Total['stim_rev']) #<- calculate prob
        DF_Total['speed'] = DF_Total['dist']/DF_Total['dura'] #<- calculate speed
        DF_Total_rows = int(DF_Total.shape[0])
        print(f'this strain/treatment has {DF_Total_rows} total taps') #<- Outputs as the second number. Check if you are missing taps!
        DF_Final = DF_Total[["time", "dura", "dist", "prob", "speed", "plate"]].copy()

    return{
            'N': Strain_N,
            'Confirm':DF_Total,
            'Final': DF_Final}
    print('done!')



def assign_taps(DF, tolerances):
    """This function inserts a taps column,
    and assigns a tap number to each row 
    based on the tolerances described prior.
    
    :param DF: any dataframe
    :type: pandas.core.frame.DataFrame
    :param tolerances: list of tuples
    :type: tuple
    
    :return: dataframe with a plate column
    :type: pandas.core.frame.DataFrame
    """
    DF['taps'] = ""
    for taps, tolerance in enumerate(tolerances): #[(99, 101), (109,111), ...]
        tap_lower,tap_upper = tolerance
        TimesInTapRange = DF['time'].between(tap_lower,tap_upper, inclusive="both")
        DF.loc[TimesInTapRange,'taps'] = taps+1 #set the tap to i where times are between



    
def insert_plates(df):   
    """This function inserts a plate column into a dataframe.
    
    :param df: any dataframe
    :type: pandas.core.frame.DataFrame
    
    :return: dataframe with a plate column
    :type: pandas.core.frame.DataFrame
    """
    df['plate']=(df['taps'] ==1).cumsum()

def Baseline_ProcessData(strain): #<- an example of a user-defined function
    strain_filelist = [x for x in filelist if strain in x] #<- goes through the list and filters for keyword
    Strain_N = len(strain_filelist) #<- N per strain, or number of plates
    Plate_N = 1
    print(f'this strain/treatment has {Strain_N} plates') #<- will output as the first number
    if Strain_N == 0:
        raise AssertionError ('{} is not a good identifier'.format(strain))
    else:
        pass
#     for f in strain_filelist:
#         DF_Total = pd.concat(pd.read_csv(f, sep=' ', skiprows = 4, header = None))
        DF_Total = pd.concat([pd.read_csv(f, sep=' ', header = None) for f in strain_filelist],
                      ignore_index=True) #<- imports your data files
#         DF_Total = DF_Total.dropna(axis = 1) #<- cleans your data
        DF_Total = DF_Total.rename( #<- more cleaning
                    {0:'Time',
                    1:'n',
                    2:'Number',
                    3:'Instantaneous Speed',
                    4:'Interval Speed',
                    5:'Bias',
                    6:'Tap',
                    7:'Puff',
                    8:'x',
                    9:'y',
                    10:'Width',
                    11:'Length',
                    12:'Area',
                    13:'Angular Speed',
                    14:'Aspect Ratio',
                    15:'Kink',
                    16:'Curve',
                    17:'Crab'}, axis=1)
        #check function here for NaN Columns
        DF_Total['plate'] = 0
        # DF_Total['prob'] = DF_Total['stim_rev']/ (DF_Total['no_rev'] + DF_Total['stim_rev']) #<- calculate prob
        # DF_Total['speed'] = DF_Total['dist']/DF_Total['dura'] #<- calculate speed
        # DF_Total_rows = int(DF_Total.shape[0])
        # print(f'this strain/treatment has {DF_Total_rows} total taps') #<- Outputs as the second number. Check if you are missing taps!
        # DF_Final = DF_Total[["time", "dura", "dist", "prob", "speed", "plate"]].copy()

    return{
            'N': Strain_N,
            'Confirm':DF_Total
            # 'Final': DF_Final
    }
    print('done!')

            
print('done step 5')

### Reading in tap data

In [ ]:
DataLists = [0]  #<- generates empty list. 0 is there to account for python's index starting at 0. 
# we want indexing to start at 1 (when I say #1 I want the first point, not the second point)

for s in list(StrainNames.values()): #<- goes through the dictionary in step 6.1 and processes data
    if not s == '':
#         threading.Thread(target=DataLists.append(ProcessData(s)['Final'])).start()
        DataLists.append(Tap_ProcessData(s)['Final']) #<- appends all data into a list of dataframes


for df in DataLists[1:]: 
    assign_taps(df, tolerances)
for df in DataLists[1:]:    
    insert_plates(df)
    
print('done step 6')

### putting tap data altogether in one big dataframe

In [ ]:
Tap_Data=pd.DataFrame()
for d in range(1,len(np.unique(genotypes))+1):
    print(d)
    Tap_Data=pd.concat([Tap_Data,
                        DataLists[d].assign(dataset=StrainNames.get(d))])

Tap_Data.reset_index(inplace=False)
print(Tap_Data)
Tap_Data.to_csv("tap_output.csv")
PD_Tap = Tap_Data
print('done step 7')

# To Read In Baseline Data:

### Setting Time Bin Experimental Parameters...

In [ ]:
bins = np.linspace(0,1200,1201) # 1 second bins (smoother lines on graph)
print(bins)


print("done step 3")

### constructing baseline data 'dat' filelist

In [ ]:
print(folder_path)
os.chdir(folder_path) #<- setting your working directory so that your images will be saved here

filelist = list() #<- empty list
for root, dirs, files in os.walk(folder_path): #<- this for loop goes through your folder 
    for name in files:
        if name.endswith('.dat'): #<- and takes out all files with a .dat (file that contains your data)
            filepath = os.path.join(root, name) #<- Notes down the file path of each data file
            filelist.append(filepath) #<- saves it into the list
    
print(filelist)
print('done step 4')

### reading in baseline data

In [ ]:
DataLists = [0] #<- generates empty list. 0 is there to account for python's index starting at 0. 
# we want indexing to start at 1 (when I say #1 I want the first point, not the second point)

for s in tqdm(StrainNames.values()): #<- goes through the dictionary in step 6.1 and processes data
    if not s == '':
        DataLists.append(Baseline_ProcessData(s)['Confirm']) #<- appends all data into a list of dataframes

print('done step 6.2')

### putting baseline data altogether in one big dataframe

In [ ]:
BaselineData=pd.concat(df.assign(dataset=StrainNames.get(i+1)) for i,df in enumerate(DataLists[1:]))
BaselineData[['Gene', 'Allele']] = BaselineData['dataset'].str.split('_', expand=True)
BaselineData['Allele']=BaselineData['Allele'].fillna('N2')
BaselineData = BaselineData.drop(columns=["plate", "Tap", "Puff", "x","y"])
# TotalConcatenated=TotalConcatenated.dropna()
# TotalConcatenated = TotalConcatenated.reset_index(drop=True)
print(BaselineData)
BaselineData.to_csv("tap_baseline_output.csv")
PD_baseline=BaselineData
print('done step 7')

### Done Importing data....

# Now onto Data Analysis

### T-Test Statistics for Baseline Data

In [ ]:
PD_baseline_last100=PD_baseline[((PD_baseline.Time<=590)&(PD_baseline.Time >=490))]
PD_baseline_last100=PD_baseline_last100.reset_index()
print(PD_baseline_last100)

In [ ]:
list=PD_baseline_last100["dataset"].unique()
print(list)

In [ ]:
def TTest_New(DatasetList, Type, DF_ref, output):
    for a in DatasetList:
        Tstat=ttest_ind(DF_ref[DF_ref.dataset == a][Type], DF_ref[DF_ref.Allele == "N2"][Type],equal_var=False)[0]
        row=[a, Tstat]
        output.loc[len(output)]=row
    print(output)

In [ ]:
PD_baseline_instantspeed_T=pd.DataFrame(columns = ["dataset","Instantaneous Speed"])
PD_baseline_intspeed_T=pd.DataFrame(columns = ["dataset","Interval Speed"])
PD_baseline_bias_T=pd.DataFrame(columns = ["dataset","Bias"])
PD_baseline_width_T=pd.DataFrame(columns = ["dataset","Width"])
PD_baseline_length_T=pd.DataFrame(columns = ["dataset","Length"])
PD_baseline_area_T=pd.DataFrame(columns = ["dataset","Area"])
PD_baseline_angularspeed_T=pd.DataFrame(columns = ["dataset","Angular Speed"])
PD_baseline_aspectratio_T=pd.DataFrame(columns = ["dataset","Aspect Ratio"])
PD_baseline_kink_T=pd.DataFrame(columns = ["dataset","Kink"])
PD_baseline_curve_T=pd.DataFrame(columns = ["dataset","Curve"])
PD_baseline_crab_T=pd.DataFrame(columns = ["dataset","Crab"])

In [ ]:
TTest_New(PD_baseline_last100["dataset"].unique(), "Instantaneous Speed", PD_baseline_last100, PD_baseline_instantspeed_T)
TTest_New(PD_baseline_last100["dataset"].unique(), "Interval Speed", PD_baseline_last100, PD_baseline_intspeed_T)
TTest_New(PD_baseline_last100["dataset"].unique(), "Bias", PD_baseline_last100, PD_baseline_bias_T)
TTest_New(PD_baseline_last100["dataset"].unique(), "Width", PD_baseline_last100, PD_baseline_width_T)
TTest_New(PD_baseline_last100["dataset"].unique(), "Length", PD_baseline_last100, PD_baseline_length_T)
TTest_New(PD_baseline_last100["dataset"].unique(), "Area", PD_baseline_last100, PD_baseline_area_T)
TTest_New(PD_baseline_last100["dataset"].unique(), "Angular Speed", PD_baseline_last100, PD_baseline_angularspeed_T)
TTest_New(PD_baseline_last100["dataset"].unique(), "Aspect Ratio", PD_baseline_last100, PD_baseline_aspectratio_T)
TTest_New(PD_baseline_last100["dataset"].unique(), "Kink", PD_baseline_last100, PD_baseline_kink_T)
TTest_New(PD_baseline_last100["dataset"].unique(), "Curve", PD_baseline_last100, PD_baseline_curve_T)
TTest_New(PD_baseline_last100["dataset"].unique(), "Crab", PD_baseline_last100, PD_baseline_crab_T)

In [ ]:
PD_baseline_Tstats=PD_baseline_instantspeed_T
PD_baseline_Tstats=PD_baseline_Tstats.join(PD_baseline_intspeed_T.iloc[:,1])
PD_baseline_Tstats=PD_baseline_Tstats.join(PD_baseline_bias_T.iloc[:,1])
PD_baseline_Tstats=PD_baseline_Tstats.join(PD_baseline_width_T.iloc[:,1])
PD_baseline_Tstats=PD_baseline_Tstats.join(PD_baseline_length_T.iloc[:,1])
PD_baseline_Tstats=PD_baseline_Tstats.join(PD_baseline_area_T.iloc[:,1])
PD_baseline_Tstats=PD_baseline_Tstats.join(PD_baseline_angularspeed_T.iloc[:,1])
PD_baseline_Tstats=PD_baseline_Tstats.join(PD_baseline_kink_T.iloc[:,1])
PD_baseline_Tstats=PD_baseline_Tstats.join(PD_baseline_curve_T.iloc[:,1])
PD_baseline_Tstats=PD_baseline_Tstats.join(PD_baseline_crab_T.iloc[:,1])
print(PD_baseline_Tstats)

In [ ]:
PD_baseline_Tstats=PD_baseline_Tstats.set_index("dataset")
print(PD_baseline_Tstats)
PD_baseline_Tstats_sort=PD_baseline_Tstats.sort_index()
print(PD_baseline_Tstats_sort)

In [ ]:
sns.set_context('notebook')
plt.figure(figsize=(14,14))
ax = sns.heatmap(data = PD_baseline_Tstats_sort, 
                 annot=False, 
                 linewidth=0.5, 
                 square=True, 
                 cmap="vlag", 
                 center=0, 
                 vmax=400,
                 vmin=-400, 
                 xticklabels=1,
                 yticklabels=1,
                 cbar_kws={"shrink": .3})
# plt.savefig(f'{StrainName}_Speed_{ISI}ISI.png_ByPlate', format='png', dpi=450)
plt.show()

### T-Test for Tap Data

In [ ]:
print(PD_tap)
PD_first_tap = PD_tap[(PD_tap.taps==1)].reset_index().drop(columns="index")
PD_last_three_taps = PD_tap[((PD_tap.taps>=28)&(PD_tap.taps<=30))].reset_index().drop(columns="index")
PD_recov_taps = PD_tap[(PD_tap.taps==31)].reset_index().drop(columns="index")
print(PD_first_tap)
print(PD_last_three_taps)
print(PD_recov_taps)

In [ ]:
PD_final_taps=PD_last_three_taps.groupby(["Gene", "Allele", "dataset", "plate"]).mean().reset_index()
print(PD_final_taps)
# print(PD_first_tap)
# print(PD_recov_taps)

In [ ]:
PD_final_taps=PD_final_taps.rename(columns={"dura": "final_dura", "prob": "final_prob", "speed": "final_speed"}, errors="raise")
PD_first_tap=PD_first_tap.rename(columns={"dura": "init_dura", "prob": "init_prob", "speed": "init_speed"}, errors="raise")
print(PD_final_taps)
print(PD_first_tap)

In [ ]:
PD_habit_levels = pd.merge(PD_first_tap, PD_final_taps, on =['dataset','plate','Gene','Allele'], how ='left')
PD_habit_levels = PD_habit_levels.dropna()
print(PD_habit_levels)

In [ ]:
PD_habit_levels['habit_dura']=PD_habit_levels.init_dura-PD_habit_levels.final_dura
PD_habit_levels['habit_prob']=PD_habit_levels.init_prob-PD_habit_levels.final_prob
PD_habit_levels['habit_speed']=PD_habit_levels.init_speed-PD_habit_levels.final_speed
PD_habit_levels=PD_habit_levels.drop(columns=['time_x','time_y','taps_x','taps_y','dist_x','dist_y'])

In [ ]:
PD_recov_taps=PD_recov_taps.rename(columns={"dura": "recov_dura", "prob": "recov_prob", "speed":"recov_speed"})
PD_habituation = pd.merge(PD_habit_levels, PD_recov_taps, on =['dataset','plate','Gene','Allele'], how ='left')
PD_habituation = PD_habituation.dropna()
print(PD_habituation)

In [ ]:
PD_habituation['recovery_dura']=PD_habituation.init_dura-PD_habituation.recov_dura
PD_habituation['recovery_prob']=PD_habituation.init_prob-PD_habituation.recov_prob
PD_habituation['recovery_speed']=PD_habituation.init_speed-PD_habituation.recov_speed
PD_habituation=PD_habituation.dropna()
print(PD_habituation)

In [ ]:
PD_recovery_dura=pd.DataFrame(columns = ["dataset","Recovery Duration"])
PD_recovery_prob=pd.DataFrame(columns = ["dataset","Recovery Probability"])
PD_recovery_speed=pd.DataFrame(columns = ["dataset","Recovery Speed"])
PD_init_dura=pd.DataFrame(columns = ["dataset","Initial Duration"])
PD_init_prob=pd.DataFrame(columns = ["dataset","Initial Probability"])
PD_init_speed=pd.DataFrame(columns = ["dataset","Initial Speed"])
PD_final_dura=pd.DataFrame(columns = ["dataset","Final Duration"])
PD_final_prob=pd.DataFrame(columns = ["dataset","Final Probability"])
PD_final_speed=pd.DataFrame(columns = ["dataset","Final Speed"])
PD_hab_dura=pd.DataFrame(columns = ["dataset","Habituation of Duration"])
PD_hab_prob=pd.DataFrame(columns = ["dataset","Habituation of Probability"])
PD_hab_speed=pd.DataFrame(columns = ["dataset","Habituation of Speed"])
print('done')

In [ ]:
print(PD_habituation["dataset"].unique())

In [ ]:
TTest_New(PD_habituation["dataset"].unique(), "recovery_dura", PD_habituation, PD_recovery_dura)
TTest_New(PD_habituation["dataset"].unique(), "recovery_prob", PD_habituation, PD_recovery_prob)
TTest_New(PD_habituation["dataset"].unique(), "recovery_speed", PD_habituation, PD_recovery_speed)
TTest_New(PD_habituation["dataset"].unique(), "init_dura", PD_habituation, PD_init_dura)
TTest_New(PD_habituation["dataset"].unique(), "init_prob", PD_habituation, PD_init_prob)
TTest_New(PD_habituation["dataset"].unique(), "init_speed", PD_habituation, PD_init_speed)
TTest_New(PD_habituation["dataset"].unique(), "final_dura", PD_habituation, PD_final_dura)
TTest_New(PD_habituation["dataset"].unique(), "final_prob", PD_habituation, PD_final_prob)
TTest_New(PD_habituation["dataset"].unique(), "final_speed", PD_habituation, PD_final_speed)
TTest_New(PD_habituation["dataset"].unique(), "habit_dura", PD_habituation, PD_hab_dura)
TTest_New(PD_habituation["dataset"].unique(), "habit_prob", PD_habituation, PD_hab_prob)
TTest_New(PD_habituation["dataset"].unique(), "habit_speed", PD_habituation, PD_hab_speed)
print("done")

In [ ]:
PD_habituation_Tstats=PD_recovery_dura
PD_habituation_Tstats=PD_habituation_Tstats.join(PD_recovery_prob.iloc[:,1])
PD_habituation_Tstats=PD_habituation_Tstats.join(PD_recovery_speed.iloc[:,1])
PD_habituation_Tstats=PD_habituation_Tstats.join(PD_init_dura.iloc[:,1])
PD_habituation_Tstats=PD_habituation_Tstats.join(PD_init_prob.iloc[:,1])
PD_habituation_Tstats=PD_habituation_Tstats.join(PD_init_speed.iloc[:,1])
PD_habituation_Tstats=PD_habituation_Tstats.join(PD_final_dura.iloc[:,1])
PD_habituation_Tstats=PD_habituation_Tstats.join(PD_final_prob.iloc[:,1])
PD_habituation_Tstats=PD_habituation_Tstats.join(PD_final_speed.iloc[:,1])
PD_habituation_Tstats=PD_habituation_Tstats.join(PD_hab_dura.iloc[:,1])
PD_habituation_Tstats=PD_habituation_Tstats.join(PD_hab_prob.iloc[:,1])
PD_habituation_Tstats=PD_habituation_Tstats.join(PD_hab_speed.iloc[:,1])
PD_habituation_Tstats=PD_habituation_Tstats.dropna()                                        
print(PD_habituation_Tstats)

In [ ]:
PD_habituation_Tstats=PD_habituation_Tstats.set_index("dataset")
print(PD_habituation_Tstats)

In [ ]:
PD_habituation_Tstats_sorted=PD_habituation_Tstats.sort_index()
print(PD_habituation_Tstats_sorted)

In [ ]:
sns.set_context('notebook')
plt.figure(figsize=(14,14))
ax = sns.heatmap(data = PD_habituation_Tstats, 
                 annot=False, 
                 linewidth=0.5, 
                 square=True, 
                 cmap="vlag", 
                 center=0, 
#                  vmax=400,
#                  vmin=-400, 
                 xticklabels=1,
                 yticklabels=1,
                 cbar_kws={"shrink": .3})
# plt.savefig(f'{StrainName}_Speed_{ISI}ISI.png_ByPlate', format='png', dpi=450)
plt.show()

# Merging all T-test statistics from both data types together

In [ ]:
PD_combined_Tstats = pd.merge(PD_baseline_Tstats, PD_habituation_Tstats, on =['dataset'], how ='left')
PD_combined_Tstats=PD_combined_Tstats.sort_index()
# PD_combined_Tstats=PD_combined_Tstats.drop(index="N2")
PD_combined_Tstats=PD_combined_Tstats.dropna()
PD_combined_Tstats_normalize=(PD_combined_Tstats-PD_combined_Tstats.mean())/PD_combined_Tstats.std()
print(PD_combined_Tstats_normalize)
print(PD_combined_Tstats_normalize.index)


In [ ]:
# print(PD_combined_Tstats_normalize[PD_combined_Tstats_normalize.index=="N2"])
# print(PD_combined_Tstats_normalize[PD_combined_Tstats_normalize.index=="N2"].squeeze())
PD_combined_Tstats_normalize_2=PD_combined_Tstats_normalize-PD_combined_Tstats_normalize[PD_combined_Tstats_normalize.index=="N2"].squeeze()
print(PD_combined_Tstats_normalize_2[PD_combined_Tstats_normalize.index=="N2"])
# print(PD_combined_Tstats_normalize[PD_combined_Tstats_normalize.index=="T20F7.3_tm2936"]-PD_combined_Tstats_normalize[PD_combined_Tstats_normalize.index=="N2"].squeeze())

In [ ]:
import matplotlib as mpl
mpl.colors.rgb_to_hsv('255,215,0')
51
240

In [ ]:
first_col=PD_combined_Tstats.pop("Area")
PD_combined_Tstats.insert(0,"Area",first_col)

first_col=PD_combined_Tstats.pop("Length")
PD_combined_Tstats.insert(0,"Length",first_col)

first_col=PD_combined_Tstats.pop("Width")
PD_combined_Tstats.insert(0,"Width",first_col)

first_col=PD_combined_Tstats_normalize_2.pop("Angular Speed")
PD_combined_Tstats_normalize_2.insert(5,"Angular Speed",first_col)

print(PD_combined_Tstats)

In [ ]:
PD_combined_Tstats=PD_combined_Tstats.rename(columns={"Habituation of Duration":"Habituation of Response Duration",
                                                     "Habituation of Probability": "Habituation of Respones Probability",
                                                     "Habituation of Speed":"Habituation of Response Speed",
                                                     "Initial Duration": "Initial Response Duration",
                                                     "Initial Probability": "Initial Response Probability",
                                                     "Initial Speed": "Initial Response Speed",
                                                     "Final Duration": "Final Response Duration",
                                                     "Final Probability": "Final Response Probability",
                                                     "Final Speed": "Final Response Speed",
                                                     "Recovery Duration": "Spontaneous Recovery of Response Duration",
                                                     "Recovery Probability": "Spontaneous Recovery of Response Probability",
                                                     "Recovery Speed": "Spontaneous Recovery of Response Speed"})
PD_combined_Tstats.to_csv('CombinedTSTATData.csv')

In [ ]:
PD_combined_Tstats_normalize_2=PD_combined_Tstats_normalize_2.rename(columns={"Habituation of Duration":"Habituation of Response Duration",
                                                                             "Habituation of Probability": "Habituation of Respones Probability",
                                                                             "Habituation of Speed":"Habituation of Response Speed",
                                                                             "Initial Duration": "Initial Response Duration",
                                                                             "Initial Probability": "Initial Response Probability",
                                                                             "Initial Speed": "Initial Response Speed",
                                                                             "Final Duration": "Final Response Duration",
                                                                             "Final Probability": "Final Response Probability",
                                                                             "Final Speed": "Final Response Speed",
                                                                             "Recovery Duration": "Spontaneous Recovery of Response Duration",
                                                                             "Recovery Probability": "Spontaneous Recovery of Response Probability",
                                                                             "Recovery Speed": "Spontaneous Recovery of Response Speed"})
print(PD_combined_Tstats_normalize_2)

In [ ]:
first_col=PD_combined_Tstats_normalize_2.pop("Area")
PD_combined_Tstats_normalize_2.insert(0,"Area",first_col)

first_col=PD_combined_Tstats_normalize_2.pop("Length")
PD_combined_Tstats_normalize_2.insert(0,"Length",first_col)

first_col=PD_combined_Tstats_normalize_2.pop("Width")
PD_combined_Tstats_normalize_2.insert(0,"Width",first_col)

first_col=PD_combined_Tstats_normalize_2.pop("Angular Speed")
PD_combined_Tstats_normalize_2.insert(5,"Angular Speed",first_col)

last_col=PD_combined_Tstats_normalize_2.pop("Spontaneous Recovery of Response Duration")
PD_combined_Tstats_normalize_2.insert(21,"Spontaneous Recovery of Response Duration",last_col)

last_col=PD_combined_Tstats_normalize_2.pop("Spontaneous Recovery of Response Probability")
PD_combined_Tstats_normalize_2.insert(21,"Spontaneous Recovery of Response Probability",last_col)

last_col=PD_combined_Tstats_normalize_2.pop("Spontaneous Recovery of Response Speed")
PD_combined_Tstats_normalize_2.insert(21,"Spontaneous Recovery of Response Speed",last_col)



print(PD_combined_Tstats_normalize_2)
PD_combined_Tstats_normalize_2.to_csv('Normalized_TSTATData.csv')

In [ ]:
sns.set_context('notebook')
plt.figure(figsize=(16,20))
ax = sns.heatmap(data = PD_combined_Tstats_normalize_2.drop(index="N2"), 
                 annot=False, 
                 linewidth=0.5, 
                 square=True, 
                 cmap="vlag", 
#                  cmap=sns.diverging_palette(55, 250, s=100, l=40,as_cmap=True), 
                 center=0, 
                 vmax=3,
                 vmin=-3, 
                 xticklabels=1,
                 yticklabels=1,
                 cbar_kws={"shrink": .2})
# plt.savefig('/Users/Joseph/Desktop/GRC_Data/Tstat_Strain_normalized.png', format='png', dpi=450, bbox_inches = "tight")
plt.savefig('Tstat_Strain_normalized.png', format='png', dpi=450, bbox_inches = "tight")
plt.show()

In [ ]:
PD_combined_Tstats_melted=PD_combined_Tstats.reset_index()
PD_combined_Tstats_melted=pd.melt(PD_combined_Tstats_melted, id_vars=['dataset'],
                                   var_name='Metric',
                                   value_name='T_score')
PD_combined_Tstats_melted_sorted=PD_combined_Tstats_melted.sort_values(by=['T_score'])
# print(PD_gene_Tstats_melted_gene)
print(PD_combined_Tstats_melted_sorted)
# PD_gene_Tstats_melted_metric=pd.melt(PD_gene_Tstats_melted_gene, id_vars=['dataset'],
#                              value_name='metric')


PD_combined_normalized_Tstats_melted=PD_combined_Tstats_normalize_2.reset_index()
PD_combined_normalized_Tstats_melted=pd.melt(PD_combined_normalized_Tstats_melted, id_vars=['dataset'],
                                   var_name='Metric',
                                   value_name='T_score')
PD_combined_normalized_Tstats_melted_sorted=PD_combined_normalized_Tstats_melted.sort_values(by=['Metric'])

In [ ]:
for x in PD_combined_Tstats_melted_sorted.Metric.unique():
    plt.figure(figsize=(12,4.8))
    ax = sns.barplot(data = PD_combined_Tstats_melted_sorted[PD_combined_Tstats_melted_sorted.Metric==f"{x}"], 
                x="dataset",
                y="T_score",
                palette=["dimgray"]).set_title(f"{x}")
    plt.xticks(rotation=90)
    plt.tick_params(axis='x', which='major', labelsize=10)
#     plt.ylim(-3,3)
    plt.savefig(f'{x}_Metrics.png', format='png', dpi=450, bbox_inches = "tight")
    plt.show()
print("done")

In [ ]:
print(PD_combined_normalized_Tstats_melted_sorted)

In [ ]:
metric_palette=["k","k","k",
                "darkgray","darkgray","darkgray","darkgray","darkgray","darkgray","darkgray",
                "lightsteelblue","lightsteelblue","lightsteelblue",
                "powderblue","powderblue","powderblue",
                "cadetblue","cadetblue","cadetblue",
                "thistle","thistle","thistle"]

sns.set_context('notebook')
for x in PD_combined_normalized_Tstats_melted_sorted.dataset.unique():
    plt.figure()
    ax = sns.barplot(data = PD_combined_normalized_Tstats_melted[PD_combined_normalized_Tstats_melted.dataset==f"{x}"], 
                x="Metric",
                y="T_score",
                palette=metric_palette).set_title(f"{x}")
    plt.ylim(-3,3)
    plt.xticks(rotation=90)
    plt.ylabel('Normalized T-score')
    plt.savefig(f'{x}_Metrics.png', format='png', dpi=450, bbox_inches = "tight")
    plt.show()
print("done")

# STOP HERE FOR NOW

## T-stat Analysis By Gene (Combining Alleles)

In [ ]:
PD_gene_baseline_width_T=pd.DataFrame(columns = ["dataset","Width"])
PD_gene_baseline_length_T=pd.DataFrame(columns = ["dataset","Length"])
PD_gene_baseline_area_T=pd.DataFrame(columns = ["dataset","Area"])
PD_gene_baseline_aspectratio_T=pd.DataFrame(columns = ["dataset","Aspect Ratio"])
PD_gene_baseline_instantspeed_T=pd.DataFrame(columns = ["dataset","Instantaneous Speed"])
PD_gene_baseline_intspeed_T=pd.DataFrame(columns = ["dataset","Interval Speed"])
PD_gene_baseline_angularspeed_T=pd.DataFrame(columns = ["dataset","Angular Speed"])
PD_gene_baseline_bias_T=pd.DataFrame(columns = ["dataset","Bias"])
PD_gene_baseline_kink_T=pd.DataFrame(columns = ["dataset","Kink"])
PD_gene_baseline_curve_T=pd.DataFrame(columns = ["dataset","Curve"])
PD_gene_baseline_crab_T=pd.DataFrame(columns = ["dataset","Crab"])


PD_gene_init_dura=pd.DataFrame(columns = ["dataset","Initial Response Duration"])
PD_gene_init_prob=pd.DataFrame(columns = ["dataset","Initial Response Probability"])
PD_gene_init_speed=pd.DataFrame(columns = ["dataset","Initial Response Speed"])
PD_gene_final_dura=pd.DataFrame(columns = ["dataset","Final Response Duration"])
PD_gene_final_prob=pd.DataFrame(columns = ["dataset","Final Response Probability"])
PD_gene_final_speed=pd.DataFrame(columns = ["dataset","Final Response Speed"])
PD_gene_hab_dura=pd.DataFrame(columns = ["dataset","Habituation of Response Duration"])
PD_gene_hab_prob=pd.DataFrame(columns = ["dataset","Habituation of Response Probability"])
PD_gene_hab_speed=pd.DataFrame(columns = ["dataset","Habituation of Response Speed"])
PD_gene_recovery_dura=pd.DataFrame(columns = ["dataset","Spontaneous Recovery of Response Duration"])
PD_gene_recovery_prob=pd.DataFrame(columns = ["dataset","Spontaneous Recovery of Response Probability"])
PD_gene_recovery_speed=pd.DataFrame(columns = ["dataset","Spontaneous Recovery of Response Speed"])

In [ ]:
PD_baseline_last100=PD_baseline_last100.dropna()
print(PD_baseline_last100)

In [ ]:
print(PD_baseline_last100.Gene.unique())
print(len(PD_baseline_last100.Gene.unique()))
type(PD_baseline_last100.Gene.unique()[1])

In [ ]:
def TTest_Gene(DatasetList, Type, DF_ref, output):
    for a in DatasetList:
        Tstat=ttest_ind(DF_ref[DF_ref.Gene == a][Type], DF_ref[DF_ref.Gene == "N2"][Type],equal_var=False)[0]
        row=[a, Tstat]
        output.loc[len(output)]=row
    print(output)

In [ ]:
print(PD_habituation.Gene.unique())
print(len(PD_habituation.Gene.unique()))
print(PD_baseline_last100.Gene.unique())
print(len(PD_baseline_last100.Gene.unique()))
PD_habituation=PD_habituation.dropna()

In [ ]:
TTest_Gene(PD_baseline_last100.Gene.unique(), 'Instantaneous Speed', PD_baseline_last100, PD_gene_baseline_instantspeed_T)
TTest_Gene(PD_baseline_last100.Gene.unique(), "Interval Speed", PD_baseline_last100, PD_gene_baseline_intspeed_T)
TTest_Gene(PD_baseline_last100.Gene.unique(), "Bias", PD_baseline_last100, PD_gene_baseline_bias_T)
TTest_Gene(PD_baseline_last100.Gene.unique(), "Width", PD_baseline_last100, PD_gene_baseline_width_T)
TTest_Gene(PD_baseline_last100.Gene.unique(), "Length", PD_baseline_last100, PD_gene_baseline_length_T)
TTest_Gene(PD_baseline_last100.Gene.unique(), "Area", PD_baseline_last100, PD_gene_baseline_area_T)
TTest_Gene(PD_baseline_last100.Gene.unique(), "Angular Speed", PD_baseline_last100, PD_gene_baseline_angularspeed_T)
TTest_Gene(PD_baseline_last100.Gene.unique(), "Aspect Ratio", PD_baseline_last100, PD_gene_baseline_aspectratio_T)
TTest_Gene(PD_baseline_last100.Gene.unique(), "Kink", PD_baseline_last100, PD_gene_baseline_kink_T)
TTest_Gene(PD_baseline_last100.Gene.unique(), "Curve", PD_baseline_last100, PD_gene_baseline_curve_T)
TTest_Gene(PD_baseline_last100.Gene.unique(), "Crab", PD_baseline_last100, PD_gene_baseline_crab_T)

TTest_Gene(PD_habituation["Gene"].unique(), "recovery_dura", PD_habituation, PD_gene_recovery_dura)
TTest_Gene(PD_habituation["Gene"].unique(), "recovery_prob", PD_habituation, PD_gene_recovery_prob)
TTest_Gene(PD_habituation["Gene"].unique(), "recovery_speed", PD_habituation, PD_gene_recovery_speed)
TTest_Gene(PD_habituation["Gene"].unique(), "init_dura", PD_habituation, PD_gene_init_dura)
TTest_Gene(PD_habituation["Gene"].unique(), "init_prob", PD_habituation, PD_gene_init_prob)
TTest_Gene(PD_habituation["Gene"].unique(), "init_speed", PD_habituation, PD_gene_init_speed)
TTest_Gene(PD_habituation["Gene"].unique(), "final_dura", PD_habituation, PD_gene_final_dura)
TTest_Gene(PD_habituation["Gene"].unique(), "final_prob", PD_habituation, PD_gene_final_prob)
TTest_Gene(PD_habituation["Gene"].unique(), "final_speed", PD_habituation, PD_gene_final_speed)
TTest_Gene(PD_habituation["Gene"].unique(), "habit_dura", PD_habituation, PD_gene_hab_dura)
TTest_Gene(PD_habituation["Gene"].unique(), "habit_prob", PD_habituation, PD_gene_hab_prob)
TTest_Gene(PD_habituation["Gene"].unique(), "habit_speed", PD_habituation, PD_gene_hab_speed)

In [ ]:
PD_gene_baseline_Tstats=PD_gene_baseline_instantspeed_T
PD_gene_baseline_Tstats=PD_gene_baseline_Tstats.join(PD_gene_baseline_intspeed_T.iloc[:,1])
PD_gene_baseline_Tstats=PD_gene_baseline_Tstats.join(PD_gene_baseline_bias_T.iloc[:,1])
PD_gene_baseline_Tstats=PD_gene_baseline_Tstats.join(PD_gene_baseline_width_T.iloc[:,1])
PD_gene_baseline_Tstats=PD_gene_baseline_Tstats.join(PD_gene_baseline_length_T.iloc[:,1])
PD_gene_baseline_Tstats=PD_gene_baseline_Tstats.join(PD_gene_baseline_area_T.iloc[:,1])
PD_gene_baseline_Tstats=PD_gene_baseline_Tstats.join(PD_gene_baseline_angularspeed_T.iloc[:,1])
PD_gene_baseline_Tstats=PD_gene_baseline_Tstats.join(PD_gene_baseline_kink_T.iloc[:,1])
PD_gene_baseline_Tstats=PD_gene_baseline_Tstats.join(PD_gene_baseline_curve_T.iloc[:,1])
PD_gene_baseline_Tstats=PD_gene_baseline_Tstats.join(PD_gene_baseline_crab_T.iloc[:,1])
PD_gene_baseline_Tstats=PD_gene_baseline_Tstats.set_index("dataset").dropna()
print(PD_gene_baseline_Tstats)
PD_gene_baseline_Tstats.to_csv("PD_gene_baseline_Tstats.csv")
PD_gene_habituation_Tstats=PD_gene_recovery_dura
PD_gene_habituation_Tstats=PD_gene_habituation_Tstats.join(PD_gene_recovery_prob.iloc[:,1])
PD_gene_habituation_Tstats=PD_gene_habituation_Tstats.join(PD_gene_recovery_speed.iloc[:,1])
PD_gene_habituation_Tstats=PD_gene_habituation_Tstats.join(PD_gene_init_dura.iloc[:,1])
PD_gene_habituation_Tstats=PD_gene_habituation_Tstats.join(PD_gene_init_prob.iloc[:,1])
PD_gene_habituation_Tstats=PD_gene_habituation_Tstats.join(PD_gene_init_speed.iloc[:,1])
PD_gene_habituation_Tstats=PD_gene_habituation_Tstats.join(PD_gene_final_dura.iloc[:,1])
PD_gene_habituation_Tstats=PD_gene_habituation_Tstats.join(PD_gene_final_prob.iloc[:,1])
PD_gene_habituation_Tstats=PD_gene_habituation_Tstats.join(PD_gene_final_speed.iloc[:,1])
PD_gene_habituation_Tstats=PD_gene_habituation_Tstats.join(PD_gene_hab_dura.iloc[:,1])
PD_gene_habituation_Tstats=PD_gene_habituation_Tstats.join(PD_gene_hab_prob.iloc[:,1])
PD_gene_habituation_Tstats=PD_gene_habituation_Tstats.join(PD_gene_hab_speed.iloc[:,1])
PD_gene_habituation_Tstats=PD_gene_habituation_Tstats.set_index("dataset").dropna()                                       
print(PD_gene_habituation_Tstats)
PD_gene_habituation_Tstats.to_csv("PD_gene_habituation_Tstats.csv")

In [ ]:
# PD_gene_Tstats_combined = pd.merge(PD_gene_baseline_Tstats, PD_gene_habituation_Tstats, on =['dataset'], how ='left')
PD_gene_Tstats_combined=PD_gene_baseline_Tstats.join(PD_gene_habituation_Tstats)
PD_gene_Tstats_combined=PD_gene_Tstats_combined.dropna()
print(PD_gene_Tstats_combined.info())
PD_gene_Tstats_combined.to_csv('PD_gene_Tstats_combined_testing.csv')
PD_gene_Tstats_combined_normalize=(PD_gene_Tstats_combined-PD_gene_Tstats_combined.mean())/PD_gene_Tstats_combined.std()
print(PD_gene_Tstats_combined_normalize)
print(PD_gene_Tstats_combined_normalize[PD_gene_Tstats_combined_normalize.index=='N2'])
PD_gene_Tstats_combined_normalize=PD_gene_Tstats_combined_normalize-PD_gene_Tstats_combined_normalize[PD_gene_Tstats_combined_normalize.index=="N2"].squeeze()
print(PD_gene_Tstats_combined_normalize)
PD_gene_Tstats_combined_normalize=PD_gene_Tstats_combined_normalize.dropna().sort_index()

PD_gene_Tstats_combined_normalize_noN2=PD_gene_Tstats_combined_normalize.drop(index="N2")
# PD_gene_Tstats_combined=PD_gene_Tstats_combined.set_index("dataset")
# PD_gene_Tstats_combined=PD_gene_Tstats_combined.sort_index()
# PD_gene_Tstats_combined=PD_gene_Tstats_combined.drop(index="N2")

# print(PD_gene_Tstats_combined_normalize)
PD_gene_Tstats_combined_normalize.rename(index={'pamn': 'pamn-1'})
print(PD_gene_Tstats_combined_normalize)
# PD_gene_Tstats_combined_normalize.to_csv('PD_gene_Tstats_combined_normalize.csv')

In [ ]:
PD_gene_Tstats_combined_normalize=PD_gene_Tstats_combined_normalize.rename(index={'pamn': 'pamn-1'})
print(PD_gene_Tstats_combined_normalize)

In [ ]:
Gene_Orthology=pd.read_csv('/Users/Joseph/Desktop/GRC_Data/Gene_Orthology.csv')
Gene_Orthology=Gene_Orthology.rename(columns={"Gene":"Orthology", "Worm Gene":"dataset"})
print(Gene_Orthology)

In [ ]:
PD_gene_Tstats_combined_normalize_joined=pd.merge(PD_gene_Tstats_combined_normalize_noN2,
                                                 Gene_Orthology,
                                                 on='dataset',
                                                 how='inner')

print(PD_gene_Tstats_combined_normalize_joined)


PD_gene_Tstats_combined=pd.merge(PD_gene_Tstats_combined_normalize_noN2,
                                                 Gene_Orthology,
                                                 on='dataset',
                                                 how='inner')

In [ ]:
PD_gene_Tstats_combined=PD_gene_Tstats_combined.rename(columns={"Orthology":"Gene"})
PD_gene_Tstats_combined=PD_gene_Tstats_combined.set_index("Gene").sort_index().drop(columns=['dataset','Unnamed: 0','Human Gene'])
print(PD_gene_Tstats_combined)

In [ ]:
PD_gene_Tstats_combined_normalize_joined=PD_gene_Tstats_combined_normalize_joined.rename(columns={"Orthology":"Gene"})
PD_gene_Tstats_combined_normalize_heatmap=PD_gene_Tstats_combined_normalize_joined.set_index("Gene").sort_index()
PD_gene_Tstats_combined_normalize_heatmap=PD_gene_Tstats_combined_normalize_heatmap.drop(columns=['dataset','Unnamed: 0','Human Gene'])

print(PD_gene_Tstats_combined_normalize_heatmap)

In [ ]:
first_col=PD_gene_Tstats_combined_normalize_heatmap.pop("Area")
PD_gene_Tstats_combined_normalize_heatmap.insert(0,"Area",first_col)

first_col=PD_gene_Tstats_combined_normalize_heatmap.pop("Length")
PD_gene_Tstats_combined_normalize_heatmap.insert(0,"Length",first_col)

first_col=PD_gene_Tstats_combined_normalize_heatmap.pop("Width")
PD_gene_Tstats_combined_normalize_heatmap.insert(0,"Width",first_col)

first_col=PD_gene_Tstats_combined_normalize_heatmap.pop("Angular Speed")
PD_gene_Tstats_combined_normalize_heatmap.insert(5,"Angular Speed",first_col)

last_col=PD_gene_Tstats_combined_normalize_heatmap.pop("Spontaneous Recovery of Response Duration")
PD_gene_Tstats_combined_normalize_heatmap.insert(21,"Spontaneous Recovery of Response Duration",last_col)

last_col=PD_gene_Tstats_combined_normalize_heatmap.pop("Spontaneous Recovery of Response Probability")
PD_gene_Tstats_combined_normalize_heatmap.insert(21,"Spontaneous Recovery of Response Probability",last_col)

last_col=PD_gene_Tstats_combined_normalize_heatmap.pop("Spontaneous Recovery of Response Speed")
PD_gene_Tstats_combined_normalize_heatmap.insert(21,"Spontaneous Recovery of Response Speed",last_col)



print(PD_gene_Tstats_combined_normalize_heatmap)

In [ ]:
sns.set_context('notebook')
plt.figure(figsize=(16,14))
ax = sns.heatmap(data = PD_gene_Tstats_combined_normalize_heatmap, 
                 annot=False, 
                 linewidth=0.5, 
                 square=True, 
                 cmap='vlag',
#                  cmap=sns.diverging_palette(55, 250, s=100, l=40,as_cmap=True), 
                 center=0, 
                 vmax=3,
                 vmin=-3, 
                 xticklabels=1,
                 yticklabels=1,
                 cbar_kws={"shrink": .2})
ax.set_ylabel('')    
plt.savefig('/Users/Joseph/Desktop/GRC_Data/Tstat_GENE_normalized_square.png', format='png', dpi=450, bbox_inches = "tight")
plt.show()

In [ ]:
sns.set_context('notebook')
g = sns.clustermap(PD_gene_Tstats_combined_normalize_heatmap,
                   cmap='vlag',
#                    cmap=sns.diverging_palette(55, 250, s=100, l=40,as_cmap=True),
                   figsize=(8,14),
                   method="ward",
                   metric="euclidean", 
                   col_cluster=False,
                   linecolor="black",
                   linewidth=0.01,
                   dendrogram_ratio=(0.2,0),
                   vmax=3,
                   center=0,
                   vmin=-3,
                   cbar_pos=(0.95, 0.16, 0.02, 0.18),
                   tree_kws=dict(linewidths=1.5))
ax = g.ax_heatmap
ax.set_ylabel('')  
g.savefig("/Users/Joseph/Desktop/GRC_Data/Cluster_edge.png", format='png', dpi=450, bbox_inches='tight')

In [ ]:
sns.set_context('notebook')
plt.figure(figsize=(20,14))
ax = sns.heatmap(data = PD_gene_Tstats_combined_normalize_heatmap, 
                 annot=False, 
                 linewidth=0.5, 
                 square=True, 
                 cmap="vlag", 
                 center=0, 
                 vmax=3,
                 vmin=-3, 
                 xticklabels=1,
                 yticklabels=1,
                 cbar_kws={"shrink": .3})
ax.set_ylabel('')    
plt.savefig('/Users/Joseph/Desktop/GRC_Data/Tstat_GENE_normalized_square_new.png', format='png', dpi=450, bbox_inches = "tight")
plt.show()

In [ ]:
first_col=PD_gene_Tstats_combined_normalize.pop("Area")
PD_gene_Tstats_combined_normalize.insert(0,"Area",first_col)

first_col=PD_gene_Tstats_combined_normalize.pop("Length")
PD_gene_Tstats_combined_normalize.insert(0,"Length",first_col)

first_col=PD_gene_Tstats_combined_normalize.pop("Width")
PD_gene_Tstats_combined_normalize.insert(0,"Width",first_col)

first_col=PD_gene_Tstats_combined_normalize.pop("Angular Speed")
PD_gene_Tstats_combined_normalize.insert(5,"Angular Speed",first_col)

last_col=PD_gene_Tstats_combined_normalize.pop("Spontaneous Recovery of Response Duration")
PD_gene_Tstats_combined_normalize.insert(21,"Spontaneous Recovery of Response Duration",last_col)

last_col=PD_gene_Tstats_combined_normalize.pop("Spontaneous Recovery of Response Probability")
PD_gene_Tstats_combined_normalize.insert(21,"Spontaneous Recovery of Response Probability",last_col)

last_col=PD_gene_Tstats_combined_normalize.pop("Spontaneous Recovery of Response Speed")
PD_gene_Tstats_combined_normalize.insert(21,"Spontaneous Recovery of Response Speed",last_col)

print(PD_gene_Tstats_combined_normalize)

In [ ]:
PD_gene_Tstats_melted=PD_gene_Tstats_combined_normalize.reset_index()
print(PD_gene_Tstats_melted)
PD_gene_Tstats_melted_gene=pd.melt(PD_gene_Tstats_melted, id_vars=['dataset'],
                                   var_name='Metric',
                                   value_name='Normalized T_score')
PD_gene_Tstats_melted_gene_sorted=PD_gene_Tstats_melted_gene.sort_values(by=['Normalized T_score'])
print(PD_gene_Tstats_melted_gene)
print(PD_gene_Tstats_melted_gene_sorted)
# PD_gene_Tstats_melted_metric=pd.melt(PD_gene_Tstats_melted_gene, id_vars=['dataset'],
#                              value_name='metric')


In [ ]:
metric_palette=["k","k","k",
                "darkgray","darkgray","darkgray","darkgray","darkgray","darkgray","darkgray",
                "lightsteelblue","lightsteelblue","lightsteelblue",
                "powderblue","powderblue","powderblue",
                "cadetblue","cadetblue","cadetblue",
                "thistle","thistle","thistle"]

In [ ]:
PD_gene_Tstats_combined_normalize_orthology_melted=PD_gene_Tstats_combined_normalize_heatmap.reset_index().rename(columns={"Spontaneous Recovery of Response Duration":"Spont. Recov. of Resp. Duration",
                                                                                                                     "Spontaneous Recovery of Response Probability":"Spont. Recov. of Resp. Probability",
                                                                                                                     "Spontaneous Recovery of Response Speed":"Spont. Recov. of Resp. Speed"})
PD_gene_Tstats_combined_normalize_orthology_melted=pd.melt(PD_gene_Tstats_combined_normalize_orthology_melted, id_vars=['Gene'],
                                   var_name='Metric',
                                   value_name='Normalized T_score')
PD_gene_Tstats_orthology_melted_gene_sorted=PD_gene_Tstats_combined_normalize_orthology_melted.sort_values(by=['Normalized T_score'])
# print(PD_gene_Tstats_melted_gene)
print(PD_gene_Tstats_orthology_melted_gene_sorted)
# PD_gene_Tstats_melted_metric=pd.melt(PD_gene_Tstats_melted_gene, id_vars=['dataset'],
#                              value_name='metric')



In [ ]:
print(PD_gene_Tstats_melted_gene.dataset.unique())
PD_gene_Tstats_combined_normalize_orthology=PD_gene_Tstats_combined_normalize_heatmap.reset_index()
print(PD_gene_Tstats_combined_normalize_orthology.Gene.unique())

In [ ]:
PD_gene_Tstats_combined_normalize_orthology_melted=PD_gene_Tstats_combined_normalize_orthology_melted.rename(columns={"Spontaneous Recovery of Response Duration":"Spont. Recov. of Resp. Duration",
                                                                                                                     "Spontaneous Recovery of Response Probability":"Spont. Recov. of Resp. Probability",
                                                                                                                     "Spontaneous Recovery of Response Speed":"Spont. Recov. of Resp. Speed"})
print(PD_gene_Tstats_combined_normalize_orthology_melted)

In [ ]:
# sns.barplot(data=PD_gene_Tstats_melted_gene[PD_gene_Tstats_melted_gene.dataset=="F53B2.5"], x="Metric", y="T_score")
# plt.xticks(rotation=90)

sns.set_context('notebook')
# # plt.figure(figsize=(14,14))
# ax = sns.barplot(data = PD_gene_Tstats_melted_gene[PD_gene_Tstats_melted_gene.dataset=="F53B2.5"], 
#                 x="Metric",
#                 y="T_score",
#                 palette=metric_palette)
# plt.xticks(rotation=90)
# # plt.savefig('/Users/Joseph/Desktop/PD_Screen_TapHab/Tstat_GENE.png', format='png', dpi=450, bbox_inches = "tight")
# # '/Users/Joseph/Desktop/PD_Screen_TapHab/Baseline100Data.csv'
# plt.show()

for x in PD_gene_Tstats_combined_normalize_orthology_melted.Gene.unique():
    plt.figure()
    ax = sns.barplot(data = PD_gene_Tstats_combined_normalize_orthology_melted[PD_gene_Tstats_combined_normalize_orthology_melted.Gene==f"{x}"], 
                x="Normalized T_score",
                y="Metric", orient='h',
                palette=metric_palette).set_title(f"{x}")
    plt.xlim(-3,3)
    plt.xlabel('Normalized T-Score')
#     plt.xticks(rotation=90)
    plt.savefig(f'/Users/Joseph/Desktop/GRC_Data/Gene_Metrics_h/{x}_Metrics.png', format='png', dpi=450, bbox_inches = "tight")
    plt.show()
print("done")

In [ ]:
# print(PD_gene_Tstats_combined)
PD_gene_Tstats_combined_melted=PD_gene_Tstats_combined.reset_index()
PD_gene_Tstats_combined_melted=pd.melt(PD_gene_Tstats_combined_melted, id_vars=['Gene'],
                                   var_name='Metric',
                                   value_name='T_score')
PD_gene_Tstats_combined_melted_sorted=PD_gene_Tstats_combined_melted.sort_values(by=['T_score'])
# print(PD_gene_Tstats_melted_gene)
print(PD_gene_Tstats_combined_melted_sorted)

In [ ]:
# sns.barplot(data=PD_gene_Tstats_melted_gene_sorted[PD_gene_Tstats_melted_gene_sorted.Metric=="Instantaneous Speed"],
#             x="dataset", y="T_score")
# plt.xticks(rotation=90)

for x in PD_gene_Tstats_combined_melted_sorted.Metric.unique():
    plt.figure(figsize=(5,10))
    ax = sns.barplot(data = PD_gene_Tstats_combined_melted_sorted[PD_gene_Tstats_combined_melted_sorted.Metric==f"{x}"], 
                x="T_score",
                y="Gene",orient='h',
                palette=["dimgray"]).set_title(f"{x}")
#     plt.xticks(rotation=90)
    plt.xlabel('T-Score')
    plt.ylabel('')
    plt.xlim(-3,3)
    plt.savefig(f'/Users/Joseph/Desktop/GRC_Data/Gene_byMetric_h/{x}_Metrics.png', format='png', dpi=450, bbox_inches = "tight")
    plt.show()
print("done")